The code below will shown how the artzone community on steemit.com is working. The brilliant idea is coming from https://steemit.com/@surfyogi and the technical development is coming from  https://steemit.com/@misrori

Artzone https://steemit.com/@artzone aim is to promote the quality post on steemit related with art! 
The code shows how to develop a voting bot which will look for a specific tag and reward the author! 

The code will run in every hour with the help of crontab. The output is also appended a log file. 

If you want to get upvote from @aztrone you have to follow the rules: 
*have artzone tag in the post
*follow @artzone
*body lenght should be greater than 300 character
*have at least on link in the post
*artzone can give just one upvote per day
*origin artwork

import the packages and print to the log the date 

In [ ]:
print('###########################################################################################################################################')
from steem import Steem
from contextlib import suppress
from steem.blockchain import Blockchain
from steem.post import Post
from steem.account import Account
from steembase.exceptions import PostDoesNotExist
import time
import sys
import pandas as pd
import datetime
from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

print("#################################################")
print(datetime.datetime.now())
print("The voting process started")

Now you can find some functions the i have created for the voting process

In [ ]:
# get the posts that are created with your tag in our case it is artzone
# I am using selenium with headless scraping.
def get_post_with_selenium(): 
    while True:
        try:
            display = Display(visible=0, size=(1920, 1080)).start()
            driver = webdriver.Firefox()
            driver.get("https://steemit.com/created/artzone")
            time.sleep(10)
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                t = list(map(lambda x : x.text, soup.select('.updated span')))
                if(('hours' in t[len(t)-1]) or ('day' in t[len(t)-1]) or ('days' in t[len(t)-1])):
                    break

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            postok = list()
            for link in soup.select('.entry-title a[href]'):
                postok.append(link['href'])
            if(len(postok)>0):
                return(postok)
                break
            else:
                print("selenium is now sleeping")
                time.sleep(300)
        except:
            print('some errror with selenium')
        finally:
            driver.close()
            display.stop()
            
# one upvote per day. in the server has 8 hour timedifference, than steemit blockchain time
def get_voted_users_last_24(user, timediff):
    account = Account(user)
    already_voted = pd.DataFrame(account.get_account_votes())
    already_voted.time = already_voted.time.astype('datetime64[ns]')
    now_data = datetime.datetime.now() - datetime.timedelta(hours=timediff)
    min_date = now_data - datetime.timedelta(hours=24)
    already_voted['author']= list(map(lambda x: x[0], already_voted.authorperm.str.split('/',1)))
    already_voted = already_voted[already_voted.time>min_date]
    already_voted = already_voted.reset_index(drop=True)
    already_voted_users =  list(already_voted['author'])
    return(already_voted_users)

# get the content of the post
def get_content(post_id):
    author = post_id[1:].split('/')[0]
    post_link = post_id[1:].split('/')[1]
    return(s.steemd.get_content(author, post_link))

# get the replies of the post
def get_replies(post_id):
    author = post_id[1:].split('/')[0]
    post_link = post_id[1:].split('/')[1]
    return(s.steemd.get_content_replies(author, post_link))

# check if artzone is already voted for the post
def check_for_votes(content):
    return('artzone' in list(map(lambda x: x['voter'], content['active_votes'])))

# get the current voting power of the bot
def get_voting_sp(user):
    t = s.steemd.get_account(user)
    last_vote = datetime.datetime.strptime(t['last_vote_time'], '%Y-%m-%dT%H:%M:%S')
    eltelt = ((datetime.datetime.now() - datetime.timedelta(hours=8)) - last_vote).total_seconds()
    current = (float(t['voting_power']) + ((10000* eltelt)/432000))
    return(current)

# check if artzone already commented on the post
def check_for_comment(replies):
    return('artzone' in list(map(lambda x:x['author'],replies)))

# check the origin of the post
def check_for_cheet(replies):
    return('cheetah' in list(map(lambda x:x['author'],replies)))




Authenticate into steemit blockchain

In [ ]:

s = Steem(keys=['<posting key>', '<active key>'])



Check if the current voting sp is greater than 85%. If not than the bot will do nothing. Hopefully in the next hour will recover. 

In [ ]:
if(get_voting_sp('artzone')<8500):
    sys.exit("Voting power is to slow")
    
  

Define the comments that the bot will post. 

In [ ]:
not_following_artzone_comment = """Dear friend, you do not appear to be following @artzone. Follow @artzone to get a valuable upvote on your quality post! """
not_quality_post_links = """To get upvote from @artzone, the post needs to have at least one link (pictures or website link). """
not_quality_post_body_length = """To get upvote from @artzone, the post needs at least 300 characters! Please describe your work in detail ;-)"""
not_using_artzone= """Dear friend! Next time also use #artzone and follow @artzone to get an upvote on your quality posts! """
length_and_link_problems="""To get an upvote from @artzone the post needs to have at least two links(pictures or website link) in the post and at least 300 characters."""
good_post = """Hello! I find your post valuable for the art community! Thanks for the great post! ARTzone is now following you! ALWAYs follow @artzone and the artzone tag, and support our artists!"""
one_upvote_per_day_commnet= '''Thanks for using #artzone tag! Artzone can give just one upvote per day!'''


Geting the posts and the voted users

In [ ]:
# get the voted users
voted_users = get_voted_users_last_24('artzone', 8)

# get the author followers
artzone_followers = Account('artzone').get_followers()

# get the post ids
sel_list = get_post_with_selenium()
sel_list =list(map(lambda x: '/'.join(x.split('/')[2:]), sel_list))


In [ ]:
#creating different list
post_to_vote = list()
comments_to_follow = list()
comments_body_length = list()
comments_links = list()
comments_oneperday= list()
comments_body_and_link = list()



The post id will be categoriezed into different category

In [ ]:

# going through and put the post into lists
for post in sel_list:
    try:
        
        t = get_content(post)
        replies = get_replies(post)

        # get body length
        body_length =len(t['body'])
        # number of links
        number_of_links = t['body'].count("http")
        # is there an answer from artzone
        answered = check_for_comment(replies)
        # cheated
        cheeted = check_for_cheet(replies)
        
        # is it upvoted by artzone
        upvoted = check_for_votes(t)
        
        # follow every artist who wrote for us
        if (t['author'] not in Account('artzone').get_following()):
            s.commit.follow(account='artzone', follow=t['author'])

        # now good post
        if(( t['author'] in artzone_followers) & (body_length>300) & (number_of_links>=1) &(answered==False) & (upvoted==False) &(cheeted ==False) ):
            # one upvote per day!    
            if( t['author'] in voted_users ):
                comments_oneperday.append(post)
                #post.reply(title="", body="Thanks for using artzone tag! Artzone can give just one upvote per day! ", author='artzone')
            else:
                post_to_vote.append(post)

        if ((t['author'] not in artzone_followers ) & (answered==False) &(cheeted ==False)):
            #post.reply(title="", body=not_following_artzone_comment, author='artzone')
            comments_to_follow.append(post)

        if((t['author'] in artzone_followers) & (body_length<300) & (number_of_links>=1) & (answered==False) &(cheeted ==False)):
            #post.reply(title="", body=not_quality_post_body_length, author='artzone')
            comments_body_length.append(post)

        if((t['author'] in artzone_followers) & (body_length>300) & (number_of_links<1) & (answered==False) &(cheeted ==False)):
            #post.reply(title="", body=not_quality_post_links, author='artzone')
            comments_links.append(post)

        if((t['author'] in artzone_followers) & (body_length<300) & (number_of_links<1) & (answered==False)&(cheeted ==False)):
            #post.reply(title="", body=length_and_link_problems, author='artzone')
            comments_body_and_link.append(post)

    except:
        print("some error")
print("#################################################")
print(datetime.datetime.now())
print("ready to vote")


The voting process

In [ ]:
#vote start
print("#################################################")
print("I have found " + str(len(post_to_vote)) + " posts to upvote"  )
print("start to upvote them")

if(len(post_to_vote)>0):
    voting_weight = 41/(len(post_to_vote))
    print("the voting_weight is "+ str(voting_weight))

    for post in post_to_vote:
        try:
            print("voting for")
            print(post)
            s.commit.vote(identifier=post,weight= voting_weight, account='artzone', )
            time.sleep(5)
            print("thanks commnet")
            s.commit.post(title="", body=good_post, author='artzone', reply_identifier=post)
            time.sleep(20)
        except:
            print("somme error during for voting")
#vote end
# ------------------------------------------------------------------------------------------------------        


In [ ]:
Comment to follow

In [ ]:
#not following start 
    
print("#################################################")
print("I have found " + str(len(comments_to_follow)) + " posts to comment to follow artzone"  )
print("start to comment them")

    

if(len(comments_to_follow)>0):
    for post in comments_to_follow:
        try:
            print("comment to follow")
            print(post)
            s.commit.post(title="", body=not_following_artzone_comment, author='artzone', reply_identifier=post)
            time.sleep(20)
        except:
            print("somme error during for not_following_artzone_comment")

#not following end
# ------------------------------------------------------------------------------------------------------        



In [ ]:
The body is to small 

In [ ]:
# body to small start
print("#################################################")
print("I have found " + str(len(comments_body_length)) + " posts to comment to body is to small"  )
print("start to comment them")


if(len(comments_body_length)>0):
    for post in comments_body_length:
        try:
            print("comment to make more characters")
            print(post)
            s.commit.post(title="", body=not_quality_post_body_length, author='artzone', reply_identifier=post)
            time.sleep(20)
        except:
            print("somme error during for not_quality_post_body_length")
# body to small ends
# ------------------------------------------------------------------------------------------------------        


In [ ]:
Not enough links

In [ ]:
# not enough link start
print("#################################################")
print("I have found " + str(len(comments_links)) + " posts to comment not enough links"  )
print("start to comment them")
    

if(len(comments_links)>0):
    for post in comments_links:
        try:
            print("comment to make more characters")
            print(post)
            s.commit.post(title="", body=not_quality_post_body_length, author='artzone', reply_identifier=post)
            time.sleep(20)
        except:
            print("somme error during for comments_links")

# not enogh link ends
# ------------------------------------------------------------------------------------------------------        


In [ ]:
One upvote per day

In [ ]:
#one vote perday start
print("#################################################")
print("I have found " + str(len(comments_oneperday)) + " posts to comment one vote per day"  )
print("start to comment them")
    
if(len(comments_oneperday)>0):
    for post in comments_oneperday:
        try:
            print("comment to one vote per day")
            print(post)
            s.commit.post(title="", body=one_upvote_per_day_commnet , author='artzone', reply_identifier=post)
            time.sleep(20)
        except:
            print("somme error during for comments_oneperday")

# one vote perday ends
# ------------------------------------------------------------------------------------------------------        


In [ ]:
Bad quality post

In [ ]:
# body and lenght start 
print("#################################################")
print("I have found " + str(len(comments_body_and_link)) + " posts to commentbody and link problems"  )
print("start to comment them")


if(len(comments_body_and_link)>0):
    for post in comments_body_and_link:
        try:
            print("comment to body and link problem")
            print(post)
            s.commit.post(title="", body=length_and_link_problems , author='artzone', reply_identifier=post)
            time.sleep(20)
        except:
            print("somme error during for length_and_link_problems")

# body and lenght ends
print("#################################################")
print(datetime.datetime.now())
print("The voting process ended")
